# Importing required packages

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset used for prototyping

All the following work must be adjusted to the correct data format, as well as converted to scripts for better handling.

In [4]:
dataset = pd.read_csv('accounts_payable_2020.csv')

In [5]:
dataset.head()

,vendor_name,transaction_date,transaction_amount,organization_code,fund,fund_desc,department,department_desc,object_code,account_description
0,JANE KENNEDY,10/07/2020,78.88,51523032,2300,Recreation Activity,515,Parks and Recreation,710500,Business Travel
1,PAMELA AUBRY,05/07/2020,41.40,13510010,1100,General,135,Municipal Court,710500,Business Travel
2,DAVID SUTHERLAND,04/15/2020,88.55,55520270,2500,Open Space,555,Open Space & Mountain Parks,710500,Business Travel
3,JAMES C. BISH JR,02/19/2020,111.09,38020010,2120,Planning and Development Svc,380,PW-Development,710515,Empl Training
4,JACOB ALBERT JACOBS,02/05/2020,852.60,51523020,2300,Recreation Activity,515,Parks and Recreation,710515,Empl Training


# Dataset adaptation

In [29]:
dataset.columns = ['company_seller_name','doc_emission_date','total_value','rm1','rm2','rm3','rm4','tag','rm5','category']

In [32]:
from random import choice
tags = ['Jantar Natal 2020','Viagem BTB','Despesas Com1','Despesas Com2','','','','','']
dataset['tag'] = dataset['tag'].apply(lambda t: choice(tags))

In [21]:
dataset = dataset.sort_values('doc_emission_date')
dataset = dataset.reset_index(drop=True)
dataset.head()

,company_seller_name,doc_emission_date,total_value,id_document,rm2,rm3,rm4,tag,rm5,category
0,PARKS & GARDENS LLC,01/01/2020,11665.30,61055020,6100,Water Utility,610,Viagem BTB,670395,Misc Purchased Services
1,KW ASSOCIATES INC,01/01/2020,76.19,38540010,2120,Planning and Development Svc,385,Jantar Natal 2020,620118,Postage and Express Mail
2,PUREWATER DYNAMICS INC,01/01/2020,77.00,31510040,1100,General,315,Despesas Com1,700495,Misc Rentals & Leases
3,WESTERN DISPOSAL INC,01/01/2020,79.05,31510070,1100,General,315,Jantar Natal 2020,670055,Solid Waste Management Fees
4,ALLIANCE FOR WATER EFFICIENCY,01/01/2020,1000.00,61010060,6100,Water Utility,610,Jantar Natal 2020,710505,"Empl Dues, Mbr, Lic, and Certs"


In [31]:
dataset = dataset.drop(['rm2','rm3','rm4','rm5','rm1'], axis=1)
dataset.head()

,company_seller_name,doc_emission_date,total_value,tag,category
0,PARKS & GARDENS LLC,01/01/2020,11665.30,Viagem BTB,Misc Purchased Services
1,KW ASSOCIATES INC,01/01/2020,76.19,Jantar Natal 2020,Postage and Express Mail
2,PUREWATER DYNAMICS INC,01/01/2020,77.00,Despesas Com1,Misc Rentals & Leases
3,WESTERN DISPOSAL INC,01/01/2020,79.05,Jantar Natal 2020,Solid Waste Management Fees
4,ALLIANCE FOR WATER EFFICIENCY,01/01/2020,1000.00,Jantar Natal 2020,"Empl Dues, Mbr, Lic, and Certs"


In [34]:
doc_id_template = "FT 20/{:06d}"

i = 0
for index, row in dataset.iterrows():
    dataset.at[index,'id_document'] = doc_id_template.format(i)
    i += 1
dataset.tail()

,company_seller_name,doc_emission_date,total_value,tag,category,id_document
24431,KEENE CONCRETE INC,10/21/2020,4766.58,,Retainage Payable,FT 20/024431
24432,ANDALE CONSTRUCTION INC,10/21/2020,6012.21,,Retainage Payable,FT 20/024432
24433,KEENE CONCRETE INC,10/21/2020,5549.51,Jantar Natal 2020,Retainage Payable,FT 20/024433
24434,KEENE CONCRETE INC,10/21/2020,327.50,,Retainage Payable,FT 20/024434
24435,"AMERESCO, INC.",10/21/2020,32433.99,Despesas Com2,Retainage Payable,FT 20/024435


In [35]:
dataset.to_csv('accounts_payable_2020_adapted.csv',index=True)

# Adjust data per Semester

## Monthly Adjusted

In [ ]:
#%%timeit

from datetime import datetime

dataset_monthly = dataset

for i, row in dataset_monthly.iterrows():
    ifor_val = datetime.strptime(row['transaction_date'], '%m/%d/%Y').month
    dataset_monthly.at[i,'month'] = ifor_val

dataset_monthly.head()

In [ ]:
#%%timeit

dataset_monthly = dataset
dataset_monthly['month'] = dataset_monthly.apply(lambda row: datetime.strptime(row['transaction_date'], '%m/%d/%Y').month, axis=1)
dataset_monthly.head()

In [101]:
#%%timeit
# More efficient way

dataset_monthly = dataset.copy()
dataset_monthly['month'] = pd.to_datetime(dataset_monthly.transaction_date)
dataset_monthly['month'] = dataset_monthly['month'].dt.strftime('%-m')
dataset_monthly.head()

,vendor_name,transaction_date,transaction_amount,nif,fund,fund_desc,department,department_desc,object_code,account_description,invoice_number,month,biannual_period
0,JANE KENNEDY,10/07/2020,78.88,51523032,2300,Recreation Activity,515,Parks and Recreation,710500,Business Travel,1000,10,1
1,PAMELA AUBRY,05/07/2020,41.40,13510010,1100,General,135,Municipal Court,710500,Business Travel,1001,5,0
2,DAVID SUTHERLAND,04/15/2020,88.55,55520270,2500,Open Space,555,Open Space & Mountain Parks,710500,Business Travel,1002,4,0
3,JAMES C. BISH JR,02/19/2020,111.09,38020010,2120,Planning and Development Svc,380,PW-Development,710515,Empl Training,1003,2,0
4,JACOB ALBERT JACOBS,02/05/2020,852.60,51523020,2300,Recreation Activity,515,Parks and Recreation,710515,Empl Training,1004,2,0


## Biannual Adjusted

May require more work, depending **if the biannual starts at different timings than the default and presumed** January 1st and July 1st.

In [85]:
dataset_biannual = dataset.copy()
dataset_biannual['biannual_period'] = pd.to_datetime(dataset_biannual.transaction_date)
dataset_biannual['biannual_period'] = dataset_biannual['biannual_period'].dt.strftime('%-m')
dataset_biannual['biannual_period'] = dataset_biannual.apply(lambda row: 0 if int(row['biannual_period']) <= 5 else 1, axis=1)
dataset_biannual.tail()

,vendor_name,transaction_date,transaction_amount,nif,fund,fund_desc,department,department_desc,object_code,account_description,invoice_number,month,biannual_period
24431,FOOD TAX REBATE,09/24/2020,87.0,110000,1100,General,0,NaN,401130,Food Tax Refunds,25431,9,1
24432,FOOD TAX REBATE,09/28/2020,2456.0,110000,1100,General,0,NaN,401130,Food Tax Refunds,25432,9,1
24433,FOOD TAX REBATE,09/29/2020,526.0,110000,1100,General,0,NaN,401130,Food Tax Refunds,25433,9,1
24434,FOOD TAX REBATE,10/01/2020,2713.0,110000,1100,General,0,NaN,401130,Food Tax Refunds,25434,10,1
24435,FOOD TAX REBATE,10/05/2020,1313.0,110000,1100,General,0,NaN,401130,Food Tax Refunds,25435,10,1


## Quartely Adjusted

Same rule as before applies. If the timelines are different than the standard, the conditions must be updated.

In [ ]:
def quaterly_adjust(row):
  m = int(row['quarterly_period'])
  if m <= 2:
    return 0
  elif m <= 5:
    return 1
  elif m <= 8:
    return 2
  else:
    return 3
  
dataset_quarterly = dataset.copy()
dataset_quarterly['quarterly_period'] = pd.to_datetime(dataset_quarterly.transaction_date)
dataset_quarterly['quarterly_period'] = dataset_quarterly['quarterly_period'].dt.strftime('%-m')
dataset_quarterly['quarterly_period'] = dataset_quarterly.apply(quaterly_adjust, axis=1)
dataset_quarterly.head()

## Yearly Adjusted

Depends on previous data, we must think first what we want to display.

# Show sector more/less profitable

In [17]:
#Show sector more/less profitable
## More profitable
print(len(dataset['department_desc'].unique().tolist()))
print(len(dataset['department'].unique().tolist()))

29
29


In [ ]:
tmp = dataset[['department_desc','transaction_amount']]
tmp = tmp.fillna('Other')
tmp

In [ ]:
tmp = tmp.groupby('department_desc').sum()
tmp.sort_values(by='transaction_amount', ascending=False)

# Earnings, Costs, Profits, Losses

In [110]:
earnings = dataset[dataset['transaction_amount'] > 0]['transaction_amount'].sum()
costs = dataset[dataset['transaction_amount'] < 0]['transaction_amount'].sum()

profits_losses = earnings - costs
print(earnings, costs, profits, sep=' , ')

197816147.59999996 , -176434.49999999997 , 197992582.09999996


# Proportion of Sectors of action

In [ ]:
dataset['fund_desc'].unique()

In [126]:
tmp = dataset[['transaction_amount','fund_desc']]
tmp = tmp.groupby('fund_desc').count()
tmp.columns = ['n_invoices']
tmp.head()
#tmp.plot.pie(y='n_invoices', autopct="%.1f%%");

,n_invoices
fund_desc,
.25 Cent Sales Tax,1206
2011 Capital Improvement Bond,5
Affordable Housing,124
Airport,139
BMPA Debt Service,1


# Top Clients

In [136]:
tmp = dataset[['vendor_name','nif']]
tmp = tmp.groupby('vendor_name').count()
tmp.columns = ['n_invoices']
tmp = tmp.sort_values(by='n_invoices', ascending=False)
tmp.head()

,n_invoices
vendor_name,
PCARD ONETIME VENDOR,2848
XCEL ENERGY,892
MCGUCKIN HARDWARE,722
HOME DEPOT USA INC,619
CENTURYLINK,355
